In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import os

In [4]:
# LDA TEST

In [5]:
#doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
#doc2 = "My father spends a lot of time driving my sister around to dance practice."
#doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
#doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
#doc5 = "Health experts say that Sugar is not good for your lifestyle."
#doc_complete = [doc1, doc2, doc3, doc4, doc5]

In [6]:
import codecs
path = "../src/data/corpus"
doc_complete = []
for i in os.listdir(path):
    if i.endswith('.txt'):
        with codecs.open(path+"/"+i, 'r', encoding='utf-8',errors='ignore') as f:
            doc_complete.append(f.read())

In [7]:
##Clean
#import nltk
#nltk.download('stopwords')
#nltk.download('wordnet')

from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete] 

In [8]:
# Importing Gensim
import gensim
from gensim.models.coherencemodel import CoherenceModel
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

In [9]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=3, id2word = dictionary, passes=50)

In [12]:
cm = CoherenceModel(model=ldamodel, corpus=doc_term_matrix, coherence='u_mass')
coherence = cm.get_coherence()

In [14]:
res = ldamodel.print_topics(num_topics=3, num_words=25)


'0.007*"peddie" + 0.006*"hbc" + 0.006*"professional" + 0.005*"accounting" + 0.005*"trade" + 0.005*"also" + 0.005*"member" + 0.004*"society" + 0.004*"accountant" + 0.004*"post" + 0.004*"good" + 0.004*"council" + 0.003*"fur" + 0.003*"cornelius" + 0.003*"year" + 0.003*"sae" + 0.003*"would" + 0.003*"outfit" + 0.003*"public" + 0.003*"simpson" + 0.003*"information" + 0.003*"record" + 0.003*"committee" + 0.003*"ibid" + 0.003*"employee"'

In [76]:
import json

index = {}
index['ciao'] = ['bye','go']
with open('data.json', 'w') as fp:
    json.dump(index, fp)

In [16]:
new_dir = 'jim'
try:
    os.mkdir(new_dir)
except:
    print -1

In [19]:
a_file = open("data/corpus/AH_1996_001.txt","r", encoding='utf-8', errors='ignore')
txt = a_file.read()

In [32]:
import re
rr = re.compile('\n')
re.sub(rr,"", txt)

'EditorialThis second issue in the New Series of the journal features articles on accounting \'s past in Scotland, China, Canada and New Zealand. Submissions of papers to date indicate that this international focus will be the hallmark of the New  Series.Professional bodies have developed means by which they deal with members \' misconduct. This has not always been the case. Walker traces the formulation of a disciplinary code by the Society of Accountants in Edinburgh (SAE) during the 1880s. Prior to this time, the social and moral controls which faced members were perceived to be sufficiently strong to prevent dishonourable behaviour. The SAE\'s reactions to adverse press coverage of the activities of members is shown to be related to the volume of adverse publicity . Further examinations of the responses of professional accounting bodies to unfavourable publicity , whether linked to the activities of individual members or of a more general nature, are  encouraged.Scorgie and Ji set 